In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=20, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args = parser.parse_args([])
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./', train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)


if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.373651
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297198
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.238961
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.199161
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.779157
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.687407
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.699854
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.524365
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.093122
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.017688
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.968281
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.086014
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.942312
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.838125
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.702017
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.683153
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.835078
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.672806
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.690350
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.612017
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.262717
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.290708
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.396469
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.166333
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.365012
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.174683
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.143885
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.154890
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.127021
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.253305
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.342319
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.160152
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.200549
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.203320
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.106005
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.279722
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.120569
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.302274
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.100437
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.489345


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.135020
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.155942
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.182258
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.107975
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.147131
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.084343
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.243224
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.297487
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.245841
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.156812
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.124441
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.042024
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.277809
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.131695
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.328592
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.117629
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.277003
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.183843
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.165058
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.078948


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.343818
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.095894
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.057673
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.114226
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.259564
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.079984
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.016316
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.080697
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.084529
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.382689
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.099211
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.237419
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.338752
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.457573
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.205394
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.079979
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.167504
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.150522
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.101485
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.094020


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.421650
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.132434
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.220899
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.164719

Test set: Average loss: 0.0466, Accuracy: 9855/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.331125
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.087588
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.212110
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.061952
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.105701
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.200227
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.125378
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.149819
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.071481
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.158747
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.176662
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.097947
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.163321
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.090343
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.151383
Train Epoch: 8 [9

...

Train Epoch: 13 [5760/60000 (10%)]	Loss: 0.190963
Train Epoch: 13 [6400/60000 (11%)]	Loss: 0.072807
Train Epoch: 13 [7040/60000 (12%)]	Loss: 0.130697
Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.197095
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.351072
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.190694
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.065025
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.070978
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.085137
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.136567
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.160825
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.069578
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.144350
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.144656
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.097007
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.162454
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.239069
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.326316
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.036936
Train Epoch: 13 [17920/60000 (30%)]	Lo

Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.072620
Train Epoch: 14 [49280/60000 (82%)]	Loss: 0.244844
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.093620
Train Epoch: 14 [50560/60000 (84%)]	Loss: 0.098161
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.216437
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.089624
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.135390
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.020958
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.091581
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.122970
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.067560
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.070296
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.194734
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.113538
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.209407
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.185384
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.191547
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.026486

Test set: Average loss: 0.0382, Accuracy: 9885/10000 (99%)

Train Epoch: 15 [0/60

Train Epoch: 16 [30720/60000 (51%)]	Loss: 0.103558
Train Epoch: 16 [31360/60000 (52%)]	Loss: 0.034018
Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.051876
Train Epoch: 16 [32640/60000 (54%)]	Loss: 0.074337
Train Epoch: 16 [33280/60000 (55%)]	Loss: 0.063179
Train Epoch: 16 [33920/60000 (57%)]	Loss: 0.401748
Train Epoch: 16 [34560/60000 (58%)]	Loss: 0.289172
Train Epoch: 16 [35200/60000 (59%)]	Loss: 0.034157
Train Epoch: 16 [35840/60000 (60%)]	Loss: 0.159493
Train Epoch: 16 [36480/60000 (61%)]	Loss: 0.153352
Train Epoch: 16 [37120/60000 (62%)]	Loss: 0.079690
Train Epoch: 16 [37760/60000 (63%)]	Loss: 0.079592
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.114947
Train Epoch: 16 [39040/60000 (65%)]	Loss: 0.012815
Train Epoch: 16 [39680/60000 (66%)]	Loss: 0.053684
Train Epoch: 16 [40320/60000 (67%)]	Loss: 0.177146
Train Epoch: 16 [40960/60000 (68%)]	Loss: 0.113898
Train Epoch: 16 [41600/60000 (69%)]	Loss: 0.058695
Train Epoch: 16 [42240/60000 (70%)]	Loss: 0.114263
Train Epoch: 16 [42880/60000 (7

Train Epoch: 18 [12800/60000 (21%)]	Loss: 0.067157
Train Epoch: 18 [13440/60000 (22%)]	Loss: 0.172326
Train Epoch: 18 [14080/60000 (23%)]	Loss: 0.100591
Train Epoch: 18 [14720/60000 (25%)]	Loss: 0.037446
Train Epoch: 18 [15360/60000 (26%)]	Loss: 0.116299
Train Epoch: 18 [16000/60000 (27%)]	Loss: 0.151034
Train Epoch: 18 [16640/60000 (28%)]	Loss: 0.086406
Train Epoch: 18 [17280/60000 (29%)]	Loss: 0.038267
Train Epoch: 18 [17920/60000 (30%)]	Loss: 0.159849
Train Epoch: 18 [18560/60000 (31%)]	Loss: 0.214069
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.042102
Train Epoch: 18 [19840/60000 (33%)]	Loss: 0.036994
Train Epoch: 18 [20480/60000 (34%)]	Loss: 0.144331
Train Epoch: 18 [21120/60000 (35%)]	Loss: 0.214018
Train Epoch: 18 [21760/60000 (36%)]	Loss: 0.191891
Train Epoch: 18 [22400/60000 (37%)]	Loss: 0.044269
Train Epoch: 18 [23040/60000 (38%)]	Loss: 0.033297
Train Epoch: 18 [23680/60000 (39%)]	Loss: 0.258018
Train Epoch: 18 [24320/60000 (41%)]	Loss: 0.080687
Train Epoch: 18 [24960/60000 (4

Train Epoch: 19 [55680/60000 (93%)]	Loss: 0.083714
Train Epoch: 19 [56320/60000 (94%)]	Loss: 0.110190
Train Epoch: 19 [56960/60000 (95%)]	Loss: 0.128505
Train Epoch: 19 [57600/60000 (96%)]	Loss: 0.030006
Train Epoch: 19 [58240/60000 (97%)]	Loss: 0.235735
Train Epoch: 19 [58880/60000 (98%)]	Loss: 0.245148
Train Epoch: 19 [59520/60000 (99%)]	Loss: 0.055595

Test set: Average loss: 0.0411, Accuracy: 9876/10000 (99%)

Train Epoch: 20 [0/60000 (0%)]	Loss: 0.041746
Train Epoch: 20 [640/60000 (1%)]	Loss: 0.162132
Train Epoch: 20 [1280/60000 (2%)]	Loss: 0.050836
Train Epoch: 20 [1920/60000 (3%)]	Loss: 0.139233
Train Epoch: 20 [2560/60000 (4%)]	Loss: 0.220794
Train Epoch: 20 [3200/60000 (5%)]	Loss: 0.019593
Train Epoch: 20 [3840/60000 (6%)]	Loss: 0.095911
Train Epoch: 20 [4480/60000 (7%)]	Loss: 0.149676
Train Epoch: 20 [5120/60000 (9%)]	Loss: 0.112909
Train Epoch: 20 [5760/60000 (10%)]	Loss: 0.294188
Train Epoch: 20 [6400/60000 (11%)]	Loss: 0.012831
Train Epoch: 20 [7040/60000 (12%)]	Loss: 0.11